In [1]:
# table output folder
out_folder='tab_out'
if (!dir.exists(out_folder)) {
  dir.create(out_folder, recursive = TRUE)
} else {
  cat("Folder already exists:", out_folder, "\n")
}
library(Seurat)
library(DOT)
library(caret)
library(dplyr)
mop= readRDS('data/merfish.rds')


Folder already exists: tab_out 


Attaching SeuratObject

Seurat v4 was just loaded with SeuratObject v5; disabling v5 assays and
validation routines, and ensuring assays work in strict v3/v4
compatibility mode

Loading required package: ggplot2

Loading required package: lattice


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union




In [5]:
out_folder='tab_out3'


In [ ]:
alpha <- 0.25
results <- list()

# Get unique slices
slices <- unique(mop$slice_id)
# Initialize an empty data.frame to store results
results_df <- data.frame(
  Nb_Gene = integer(),
  Slice_ID = character(),
  Accuracy = numeric(),
  F1_Score = numeric(),
  stringsAsFactors = FALSE
)

# Loop over the number of genes
for (nb_gene in c(50, 100, 150, 200)) {
    
  for (slice in slices) {
      print(slice)
    # Subset data for the target and reference
    mop_sub <- subset(mop, subset = slice_id == slice)
    mop_ref <- subset(mop, subset = slice_id != slice)
    mop_ref <- NormalizeData(mop_ref)
    mop_ref <- FindVariableFeatures(mop_ref)
    mop_ref <- ScaleData(mop_ref)
    mop_ref <- RunPCA(mop_ref)
#     mop_ref <- FindNeighbors(mop_ref, dims = 1:30)

    # Select only the first nb_genes for the target dataset

    # Introduce noise to the target
    beta_values <- 1 + runif(ncol(mop_sub), min = -alpha, max = alpha)
    mop_sub@assays$RNA@counts <- t(t(as.matrix(mop_sub@assays$RNA@counts) * beta_values))
    mop_sub_sub <- mop_sub[1:nb_gene, ]

    # Get coordinates
    coord_sub_sub <- mop_sub_sub@meta.data[, c("x", "y")]
    mop_sub_sub <- NormalizeData(mop_sub_sub)
    pancreas.anchors <- FindTransferAnchors(reference = mop_ref, query = mop_sub_sub, dims = 1:30,
        reference.reduction = "pca")
    predictions <- TransferData(anchorset = pancreas.anchors, refdata = mop_ref$subclass, dims = 1:30)
    mop_sub_sub <- AddMetaData(mop_sub_sub, metadata = predictions)

    # Compute accuracy
    predictions <- factor(mop_sub_sub$predicted.id)
    true_labels=mop_sub_sub$subclass
    missing_classes <- setdiff(levels(predictions), levels(true_labels))
    true_labels <- factor(true_labels, levels = c(levels(true_labels), missing_classes))
    common_levels <- union(levels(true_labels), levels(predictions))
    true_labels <- factor(true_labels, levels = common_levels)
    predictions <- factor(predictions, levels = common_levels)
      
    accuracy<- sum(true_labels == predictions) / length(predictions)
    conf_matrix <- table(Predicted = predictions, Actual = true_labels)
    precision <- diag(conf_matrix) / colSums(conf_matrix)  # Precision for each class
    recall <- diag(conf_matrix) / rowSums(conf_matrix)  # Recall for each class
    f1_scores <- 2 * (precision * recall) / (precision + recall)
    f1_scores[is.na(f1_scores)]=0
    f1_score <- mean(f1_scores)
    print(accuracy)
    print(f1_score)
     
    results_df <- results_df %>%
      add_row(
        Nb_Gene = nb_gene,
        Slice_ID = slice,
        Accuracy = accuracy,
        F1_Score = f1_score
      )      
      
  }}


[1] "mouse2_slice31"


Centering and scaling data matrix

PC_ 1 
Positive:  Syndig1, Epha7, Satb2, Cdh12, Wipf3, Grm8, Nr4a1, Grm1, Unc5d, Col25a1 
	   Shisa9, Prdm8, Cbln2, Prss12, Flrt3, C1ql3, Rgs6, Iqgap2, Slc30a3, Cntnap5b 
	   Ptprt, Prr16, Trpc4, Sertm1, Fam19a2, Adra1b, Olfm3, Calb2, Tenm3, Rfx3 
Negative:  Vip, Gad2, Cobll1, Cldn5, Flt1, Slc17a7, Rab3b, Unc5b, Gad1, Plekhg3 
	   Pvalb, Timp3, Tac2, Sst, Pdlim5, Bgn, Calb1, Igf2, Prox1, Opalin 
	   Cd14, Daam2, Sox10, Gfap, Cspg4, Cnr1, Nr2f2, Ptprm, Vtn, Pxdc1 
PC_ 2 
Positive:  Lamp5, Cux2, Unc5d, Slc30a3, Prr16, Fam19a2, Prdm8, Slc17a6, Cbln4, C1ql3 
	   Rspo1, Slc17a7, Rorb, Otof, Tcap, Calb1, Cdh12, Ankfn1, Brinp3, Syndig1 
	   Ccbe1, Cntnap5b, Sertm1, Ntng2, Ptprk, Gpc6, Pld5, Ptprt, Pcdh8, Bdnf 
Negative:  Sema5a, Syt6, Foxp2, Bcl11b, Ano4, Adamts4, Spon1, Cdh9, Prss23, Sulf2 
	   Fezf2, Rnf152, Ptpru, Dscaml1, Sulf1, Tmem163, Opalin, Sox10, Prox1, Tox 
	   Cdh13, Plekhg3, Lsp1, Rxfp1, Sema3e, Plcxd3, Hs3st5, Slc25a13, Myh14, Unc5b 
PC_ 3 
Pos

[1] 0.7201767
[1] 0.4709109
[1] "mouse2_slice160"


Centering and scaling data matrix

PC_ 1 
Positive:  Syndig1, Epha7, Satb2, Cdh12, Wipf3, Grm8, Nr4a1, Grm1, Unc5d, Col25a1 
	   Shisa9, Prdm8, Cbln2, Flrt3, Prss12, C1ql3, Rgs6, Iqgap2, Slc30a3, Cntnap5b 
	   Ptprt, Prr16, Trpc4, Sertm1, Fam19a2, Adra1b, Olfm3, Calb2, Tenm3, Rfx3 
Negative:  Vip, Gad2, Cobll1, Cldn5, Flt1, Slc17a7, Rab3b, Unc5b, Gad1, Plekhg3 
	   Pvalb, Timp3, Tac2, Sst, Pdlim5, Bgn, Calb1, Igf2, Prox1, Opalin 
	   Cd14, Daam2, Gfap, Sox10, Cspg4, Cnr1, Nr2f2, Ptprm, Vtn, Pxdc1 
PC_ 2 
Positive:  Lamp5, Cux2, Unc5d, Slc30a3, Prr16, Fam19a2, Prdm8, Slc17a6, Cbln4, C1ql3 
	   Slc17a7, Rspo1, Rorb, Otof, Tcap, Calb1, Cdh12, Ankfn1, Brinp3, Syndig1 
	   Ccbe1, Cntnap5b, Sertm1, Ntng2, Ptprk, Gpc6, Pld5, Ptprt, Pcdh8, Bdnf 
Negative:  Sema5a, Syt6, Foxp2, Bcl11b, Ano4, Adamts4, Spon1, Cdh9, Sulf2, Prss23 
	   Fezf2, Rnf152, Ptpru, Dscaml1, Sulf1, Tmem163, Opalin, Sox10, Prox1, Tox 
	   Cdh13, Plekhg3, Sema3e, Rxfp1, Lsp1, Plcxd3, Slc25a13, Unc5b, Hs3st5, Myh14 
PC_ 3 
Pos

[1] 0.71048
[1] 0.5366327
[1] "mouse2_slice109"


...

Centering and scaling data matrix

PC_ 1 
Positive:  Syndig1, Epha7, Satb2, Cdh12, Wipf3, Grm8, Nr4a1, Grm1, Unc5d, Col25a1 
	   Shisa9, Prdm8, Cbln2, Prss12, Flrt3, C1ql3, Rgs6, Iqgap2, Slc30a3, Cntnap5b 
	   Ptprt, Prr16, Trpc4, Sertm1, Fam19a2, Adra1b, Olfm3, Calb2, Tenm3, Rfx3 
Negative:  Vip, Gad2, Cobll1, Cldn5, Flt1, Slc17a7, Rab3b, Unc5b, Gad1, Plekhg3 
	   Timp3, Pvalb, Sst, Tac2, Bgn, Pdlim5, Calb1, Igf2, Prox1, Cd14 
	   Opalin, Daam2, Cspg4, Gfap, Cnr1, Sox10, Nr2f2, Ptprm, Vtn, Pxdc1 
PC_ 2 
Positive:  Lamp5, Cux2, Unc5d, Slc30a3, Prr16, Fam19a2, Prdm8, Slc17a6, Cbln4, C1ql3 
	   Rspo1, Slc17a7, Rorb, Otof, Tcap, Calb1, Cdh12, Ankfn1, Brinp3, Ccbe1 
	   Syndig1, Sertm1, Cntnap5b, Ntng2, Ptprk, Gpc6, Pld5, Pcdh8, Ptprt, Bdnf 
Negative:  Sema5a, Syt6, Foxp2, Bcl11b, Ano4, Adamts4, Spon1, Cdh9, Prss23, Sulf2 
	   Fezf2, Rnf152, Ptpru, Dscaml1, Sulf1, Tmem163, Opalin, Sox10, Prox1, Tox 
	   Cdh13, Plekhg3, Lsp1, Rxfp1, Sema3e, Plcxd3, Hs3st5, Slc25a13, Myh14, Unc5b 
PC_ 3 
Pos

[1] 0.8231604
[1] 0.7037678
[1] "mouse2_slice209"


Centering and scaling data matrix

PC_ 1 
Positive:  Vip, Gad2, Cobll1, Cldn5, Flt1, Slc17a7, Rab3b, Unc5b, Gad1, Plekhg3 
	   Timp3, Pvalb, Tac2, Sst, Pdlim5, Bgn, Calb1, Igf2, Prox1, Cd14 
	   Opalin, Daam2, Cspg4, Gfap, Cnr1, Sox10, Nr2f2, Ptprm, Vtn, Pxdc1 
Negative:  Syndig1, Epha7, Satb2, Cdh12, Wipf3, Grm8, Nr4a1, Grm1, Unc5d, Col25a1 
	   Shisa9, Prdm8, Cbln2, C1ql3, Flrt3, Prss12, Rgs6, Iqgap2, Slc30a3, Cntnap5b 
	   Ptprt, Prr16, Trpc4, Sertm1, Fam19a2, Adra1b, Calb2, Olfm3, Tenm3, Rfx3 
PC_ 2 
Positive:  Lamp5, Cux2, Unc5d, Slc30a3, Prr16, Fam19a2, Prdm8, Slc17a6, Cbln4, C1ql3 
	   Rspo1, Slc17a7, Rorb, Otof, Tcap, Calb1, Cdh12, Ankfn1, Brinp3, Syndig1 
	   Ccbe1, Cntnap5b, Sertm1, Ptprk, Ntng2, Gpc6, Pld5, Ptprt, Pcdh8, Bdnf 
Negative:  Sema5a, Syt6, Foxp2, Bcl11b, Ano4, Adamts4, Spon1, Cdh9, Prss23, Sulf2 
	   Fezf2, Rnf152, Ptpru, Dscaml1, Sulf1, Tmem163, Opalin, Sox10, Tox, Prox1 
	   Cdh13, Plekhg3, Rxfp1, Sema3e, Lsp1, Plcxd3, Hs3st5, Slc25a13, Myh14, Unc5b 
PC_ 3 
Pos

[1] 0.8617501
[1] 0.7558219
[1] "mouse1_slice50"


Centering and scaling data matrix

PC_ 1 
Positive:  Vip, Gad2, Cobll1, Cldn5, Flt1, Slc17a7, Rab3b, Unc5b, Gad1, Plekhg3 
	   Timp3, Pvalb, Tac2, Pdlim5, Sst, Bgn, Calb1, Igf2, Prox1, Opalin 
	   Cd14, Daam2, Gfap, Sox10, Cspg4, Cnr1, Nr2f2, Ptprm, Vtn, Pxdc1 
Negative:  Syndig1, Epha7, Satb2, Cdh12, Wipf3, Grm8, Nr4a1, Grm1, Unc5d, Col25a1 
	   Shisa9, Prdm8, Cbln2, Prss12, Flrt3, C1ql3, Rgs6, Iqgap2, Slc30a3, Cntnap5b 
	   Ptprt, Prr16, Trpc4, Sertm1, Fam19a2, Adra1b, Olfm3, Calb2, Tenm3, Rfx3 
PC_ 2 
Positive:  Lamp5, Cux2, Unc5d, Slc30a3, Fam19a2, Prr16, Prdm8, Slc17a6, Cbln4, C1ql3 
	   Rspo1, Slc17a7, Rorb, Otof, Tcap, Calb1, Cdh12, Ankfn1, Brinp3, Syndig1 
	   Ccbe1, Cntnap5b, Sertm1, Ntng2, Ptprk, Gpc6, Pld5, Ptprt, Pcdh8, Bdnf 
Negative:  Sema5a, Syt6, Foxp2, Bcl11b, Ano4, Adamts4, Spon1, Cdh9, Sulf2, Prss23 
	   Fezf2, Rnf152, Ptpru, Dscaml1, Sulf1, Tmem163, Opalin, Sox10, Prox1, Cdh13 
	   Tox, Plekhg3, Lsp1, Sema3e, Rxfp1, Plcxd3, Hs3st5, Slc25a13, Myh14, Unc5b 
PC_ 3 
Pos

In [6]:
write.csv(results_df,paste0(out_folder,'/CellAnnotation_MERFISH_seurat.csv'))

In [7]:
results_df

Nb_Gene,Slice_ID,Accuracy,F1_Score
<dbl>,<chr>,<dbl>,<dbl>
50,mouse2_slice31,0.7201767,0.4709109
50,mouse2_slice160,0.7104800,0.5366327
50,mouse2_slice109,0.7039310,0.5737658
50,mouse1_slice71,0.6780310,0.5432062
50,mouse2_slice219,0.7192737,0.5602602
50,mouse2_slice289,0.7129143,0.5339513
50,mouse1_slice313,0.7015306,0.5428937
50,mouse2_slice309,0.7284587,0.5128412
50,mouse1_slice153,0.6808287,0.5505434


In [ ]:

# alpha <- 0.25
# results <- list()

# # Get unique slices
# slices <- unique(mop$slice_id)

# # Loop over the number of genes
# for (nb_gene in c(50, 100, 150, 200)) {
#   acc_ <- list()

#   for (slice in slices) {
#     print(slice)
#     # Subset data for the target and reference
#     mop_sub <- subset(mop, subset = slice_id == slice)
#     mop_ref <- subset(mop, subset = slice_id != slice)
#     mop_ref <- NormalizeData(mop_ref)
#     mop_ref <- FindVariableFeatures(mop_ref)
#     mop_ref <- ScaleData(mop_ref)
#     mop_ref <- RunPCA(mop_ref)
# #     mop_ref <- FindNeighbors(mop_ref, dims = 1:30)

#     # Select only the first nb_genes for the target dataset

#     # Introduce noise to the target
#     beta_values <- 1 + runif(ncol(mop_sub), min = -alpha, max = alpha)
#     mop_sub@assays$RNA@counts <- t(t(as.matrix(mop_sub@assays$RNA@counts) * beta_values))
#     mop_sub_sub <- mop_sub[1:nb_gene, ]

#     # Get coordinates
#     coord_sub_sub <- mop_sub_sub@meta.data[, c("x", "y")]
#     mop_sub_sub <- NormalizeData(mop_sub_sub)
#     pancreas.anchors <- FindTransferAnchors(reference = mop_ref, query = mop_sub_sub, dims = 1:30,
#         reference.reduction = "pca")
#     predictions <- TransferData(anchorset = pancreas.anchors, refdata = mop_ref$subclass, dims = 1:30)
#     mop_sub_sub <- AddMetaData(mop_sub_sub, metadata = predictions)

#     # Compute accuracy
#     cellt <- mop_sub_sub$predicted.id
#     acc_[[slice]] <- sum(mop_sub_sub$subclass == cellt) / length(cellt)
#     print(acc_[[slice]])      

#   }

#   # Store results for the current number of genes
#   results[[as.character(nb_gene)]] <- list(
#     accuracy = acc_  )
# }


In [ ]:
# saveRDS(results,'seurat.rds')